In [1]:
#Importing Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import pybaseball as pyb
from datetime import datetime
from mega import mega_df
from mega import mega_df_cleaned
from functions import team_batting_avg
import re


In [3]:
#Viewing the mega dataframe
df= mega_df
df.tail()

,Gm#,Date,Tm,H/A,Opp,W/L,R,RA,Inn,W-L,...,Save,Time,D/N,Attendance,cLI,Streak,Orig. Scheduled,Year,Full_Date,Formatted_Date
1837,158,Wednesday Sep 25,COL,H,STL,8:40 pm,8:40 pm,8:40 pm,Game Preview and Matchups,Game Preview and Matchups,...,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,NaN,NaN,NaN,2024,"Wednesday Sep 25, 2024",2024-09-25
1838,159,Thursday Sep 26,COL,H,STL,3:10 pm,3:10 pm,3:10 pm,Game Preview and Matchups,Game Preview and Matchups,...,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,NaN,NaN,NaN,2024,"Thursday Sep 26, 2024",2024-09-26
1839,160,Friday Sep 27,COL,H,LAD,8:10 pm,8:10 pm,8:10 pm,Game Preview and Matchups,Game Preview and Matchups,...,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,NaN,NaN,NaN,2024,"Friday Sep 27, 2024",2024-09-27
1840,161,Saturday Sep 28,COL,H,LAD,8:10 pm,8:10 pm,8:10 pm,Game Preview and Matchups,Game Preview and Matchups,...,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,NaN,NaN,NaN,2024,"Saturday Sep 28, 2024",2024-09-28
1841,162,Sunday Sep 29,COL,H,LAD,3:10 pm,3:10 pm,3:10 pm,Game Preview and Matchups,Game Preview and Matchups,...,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,NaN,NaN,NaN,2024,"Sunday Sep 29, 2024",2024-09-29


In [25]:
df.head()

,Gm#,Date,Tm,H/A,Opp,W/L,R,RA,Inn,W-L,...,Time,D/N,Attendance,cLI,Streak,Orig. Scheduled,Year,Month,fulldate,FormattedDate
0,1,Monday Apr 1,COL,@,MIL,L-wo,4,5,10.0,0-1,...,3:13,D,45781,0.93,-,NaN,2013,NaN,"Monday Apr 1, 2013",2013-04-01
1,2,Tuesday Apr 2,COL,@,MIL,W,8,4,NaN,1-1,...,3:22,N,24753,0.93,+,NaN,2013,NaN,"Tuesday Apr 2, 2013",2013-04-02
2,3,Wednesday Apr 3,COL,@,MIL,W,7,3,NaN,2-1,...,3:31,N,25766,0.95,++,NaN,2013,NaN,"Wednesday Apr 3, 2013",2013-04-03
3,4,Friday Apr 5,COL,H,SDP,W,5,2,NaN,3-1,...,2:45,D,49077,1.08,+++,NaN,2013,NaN,"Friday Apr 5, 2013",2013-04-05
4,5,Saturday Apr 6,COL,H,SDP,W,6,3,NaN,4-1,...,2:54,N,31133,1.12,++++,NaN,2013,NaN,"Saturday Apr 6, 2013",2013-04-06


In [32]:
game = pyb.statcast(start_dt=date_list[0], end_dt=date_list[0],team="COL")
game

This is a large query, it may take a moment to complete


100%|██████████| 1/1 [00:02<00:00,  2.05s/it]


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length
82,ST,2013-04-01,84.6,-3.69,5.74,"Ottavino, Adam",518960,493603,sac_fly,hit_into_play,...,5,5,4,<NA>,<NA>,<NA>,0.171,-0.317,<NA>,<NA>
85,ST,2013-04-01,84.8,-3.72,5.61,"Ottavino, Adam",518960,493603,NaN,called_strike,...,4,4,4,<NA>,<NA>,<NA>,0.0,-0.11,<NA>,<NA>
88,FF,2013-04-01,93.8,-3.69,5.57,"Ottavino, Adam",133380,493603,walk,ball,...,4,4,4,<NA>,<NA>,<NA>,0.126,0.415,<NA>,<NA>
92,FF,2013-04-01,94.1,-3.58,5.69,"Ottavino, Adam",133380,493603,NaN,ball,...,4,4,4,<NA>,<NA>,<NA>,0.0,0.17,<NA>,<NA>
97,ST,2013-04-01,84.4,-3.67,5.7,"Ottavino, Adam",133380,493603,NaN,called_strike,...,4,4,4,<NA>,<NA>,<NA>,0.0,-0.12,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,SI,2013-04-01,90.7,-2.09,6.1,"Chacín, Jhoulys",493114,468504,walk,ball,...,0,0,0,<NA>,<NA>,<NA>,0.036,0.215,<NA>,<NA>
137,SI,2013-04-01,90.3,-1.92,6.18,"Chacín, Jhoulys",493114,468504,NaN,ball,...,0,0,0,<NA>,<NA>,<NA>,0.0,0.106,<NA>,<NA>
141,SI,2013-04-01,90.4,-1.94,6.21,"Chacín, Jhoulys",493114,468504,NaN,ball,...,0,0,0,<NA>,<NA>,<NA>,0.0,0.043,<NA>,<NA>
145,SI,2013-04-01,90.8,-2.04,6.21,"Chacín, Jhoulys",493114,468504,NaN,called_strike,...,0,0,0,<NA>,<NA>,<NA>,0.0,-0.04,<NA>,<NA>


In [3]:
mega_df.head()
funcions

,Gm#,Date,Tm,H/A,Opp,W/L,R,RA,Inn,W-L,...,Loss,Save,Time,D/N,Attendance,cLI,Streak,Orig. Scheduled,Year,Formatted_Date
0,1,Monday Apr 1,COL,@,MIL,L-wo,4,5,10.0,0-1,...,Ottavino,NaN,3:13,D,45781,0.93,-,NaN,2013,2013-04-01
1,2,Tuesday Apr 2,COL,@,MIL,W,8,4,NaN,1-1,...,Gonzalez,Betancourt,3:22,N,24753,0.93,+,NaN,2013,2013-04-02
2,3,Wednesday Apr 3,COL,@,MIL,W,7,3,NaN,2-1,...,Peralta,NaN,3:31,N,25766,0.95,++,NaN,2013,2013-04-03
3,4,Friday Apr 5,COL,H,SDP,W,5,2,NaN,3-1,...,Marquis,Betancourt,2:45,D,49077,1.08,+++,NaN,2013,2013-04-05
4,5,Saturday Apr 6,COL,H,SDP,W,6,3,NaN,4-1,...,Ross,Betancourt,2:54,N,31133,1.12,++++,NaN,2013,2013-04-06


In [24]:
col_concat_dict={"COL_atbats":"COL_atbats_column.csv",
                 "COL_BA":"COL_ba_column.csv",
                 "COL_Hits":"COL_hits_column.csv",
                 "COL_HR":"COL_hr_column.csv",
                 "COL_KK":"COL_kk_column.csv",
                 "COL_OBP":"COL_obp_column.csv",
                 "COL_walks":"COL_walks_column.csv"}

In [35]:
col_df_list=[]
def make_df_cols(name,path):
    """
    Reads a CSV file into a DataFrame and appends it to the global list col_df_list.

    Args:
        name (str): The name associated with the DataFrame (not used in the function).
        path (str): The filename of the CSV file to read.
    """
    path_first="../Project 2/progress_dataframes/"
    path_second=path
    full_path=path_first + path_second
    df=pd.read_csv(full_path,index_col=0)
    col_df_list.append(df)
    

In [36]:
for key,value in col_concat_dict.items():
    result=make_df_cols(key,value)
    
test_df = col_df_list[0].join(col_df_list[1:], how='outer')

In [37]:
col_df_list

[      COL_at_bats
 0              31
 1              32
 2              34
 3              33
 4              33
 ...           ...
 1837           -1
 1838           -1
 1839           -1
 1840           -1
 1841           -1
 
 [1780 rows x 1 columns],
         COL_ba
 0     0.258065
 1     0.281250
 2     0.264706
 3     0.212121
 4     0.242424
 ...        ...
 1837 -1.000000
 1838 -1.000000
 1839 -1.000000
 1840 -1.000000
 1841 -1.000000
 
 [1780 rows x 1 columns],
       COL_hits
 0            8
 1            9
 2            9
 3            7
 4            8
 ...        ...
 1837        -1
 1838        -1
 1839        -1
 1840        -1
 1841        -1
 
 [1780 rows x 1 columns],
       COL_hr
 0          1
 1          1
 2          1
 3          0
 4          0
 ...      ...
 1837      -1
 1838      -1
 1839      -1
 1840      -1
 1841      -1
 
 [1780 rows x 1 columns],
       COL_kk
 0          8
 1          5
 2          5
 3          8
 4          4
 ...      ...
 1837     

In [38]:
test_df

,COL_at_bats,COL_ba,COL_hits,COL_hr,COL_kk,COL_obp,COL_walks
0,31,0.258065,8,1,8,0.351351,4
1,32,0.281250,9,1,5,0.342857,3
2,34,0.264706,9,1,5,0.305556,2
3,33,0.212121,7,0,8,0.235294,1
4,33,0.242424,8,0,4,0.305556,3
...,...,...,...,...,...,...,...
1837,-1,-1.000000,-1,-1,-1,-1.000000,-1
1838,-1,-1.000000,-1,-1,-1,-1.000000,-1
1839,-1,-1.000000,-1,-1,-1,-1.000000,-1
1840,-1,-1.000000,-1,-1,-1,-1.000000,-1


In [41]:
mega_concat_df=pd.concat([mega_df_cleaned,test_df],axis=1)
mega_concat_df.head()

,Gm#,Date,Tm,H/A,Opp,W/L,R,RA,Inn,W-L,Rank,GB,Win,Loss,Save,Time,D/N,Attendance,cLI,Streak,Orig. Scheduled,Year,Full_Date,Formatted_Date,COL_at_bats,COL_ba,COL_hits,COL_hr,COL_kk,COL_obp,COL_walks
0,1,Monday Apr 1,COL,@,MIL,L-wo,4,5,10.0,0-1,3,1.0,Henderson,Ottavino,NaN,3:13,D,45781,0.93,-,NaN,2013,"Monday Apr 1, 2013",2013-04-01,31,0.258065,8,1,8,0.351351,4
1,2,Tuesday Apr 2,COL,@,MIL,W,8,4,NaN,1-1,1,Tied,Escalona,Gonzalez,Betancourt,3:22,N,24753,0.93,+,NaN,2013,"Tuesday Apr 2, 2013",2013-04-02,32,0.281250,9,1,5,0.342857,3
2,3,Wednesday Apr 3,COL,@,MIL,W,7,3,NaN,2-1,1,Tied,Nicasio,Peralta,NaN,3:31,N,25766,0.95,++,NaN,2013,"Wednesday Apr 3, 2013",2013-04-03,34,0.264706,9,1,5,0.305556,2
3,4,Friday Apr 5,COL,H,SDP,W,5,2,NaN,3-1,1,Tied,Francis,Marquis,Betancourt,2:45,D,49077,1.08,+++,NaN,2013,"Friday Apr 5, 2013",2013-04-05,33,0.212121,7,0,8,0.235294,1
4,5,Saturday Apr 6,COL,H,SDP,W,6,3,NaN,4-1,1,Tied,Garland,Ross,Betancourt,2:54,N,31133,1.12,++++,NaN,2013,"Saturday Apr 6, 2013",2013-04-06,33,0.242424,8,0,4,0.305556,3


In [42]:
from mega import mega_concat_df

ImportError: cannot import name 'mega_concat_df' from 'mega' (c:\Users\cassy\OneDrive\ai bootcamp\Homework\rockies_moneyball_part-deux\Project 2\mega.py)

### Going Through Linear and Logistic Regression Models